_________

# Land Cover Classification for use in the CAPRA Model 

## Data Acquisition


_________

### Learning Objectives

In this tutorial you will learn how to search, find and visualize remotely sensed imagery from different sources like Google Earth Engine and the USGS Global Visualization Viewer (GloVis).

### Remote Sensing Data in the CAPRA model

The Central American Probabilistic Risk Assessment model for flood hazards is a secondary hazard model that requires spatially explicit runoff factors (Fig. 1, Tbl. 1).  Spatially explicit runoff values are determined from water evaporation and infiltration rates, which are a function of the land cover, land use and the soil characteristics of a location.  Hence, to produce a runoff factor (N) map it is essential to have a land cover map.

<img src="./imgs/CAPRAflood.PNG">

The purpose of this tutorial is to derive a land cover map from satellite imagery using a classification method called recursive partitioning.  The procedure for the classification process is outlined in Figure 2. 

The tutorial will use the open source software Google Earth Engine (GEE) and its Python API. We will cover common tasks for data loading, image visualization and processing. The main steps of the classification process we developed for this tutorial with relevant software indicated in () are:

(1) Installation of Anaconda distributions including Spyder and Jupyter notebook plug-in<br>
(2) Classification Scheme Definition<br> 
(3) Remote Sensing Process<br>
(4) Free Data: Sensors<br>
(5) Acquisition and Processing Options (Exploration: GloVis, USGS-EE; Download: USGS-EE, ESPA; Cloud Processing: GEE)<br>
(6) Image Evaluation and Visualization (GEE Python API)<br>
(7) Radiometric Correction (GEE Python API)<br>
(8) Derivation of Additional Information from Reflectance Image (GEE Python API)<br>
(9) Evaluation of Variable Set and Classification Algorithm<br> - Digitization of Reference Locations for Training Set (Google Earth)<br> - Evaluation of Variable Set and Classification Algorithm (GEE Python API)<br> - Classification and evaluation of classifiers (GEE Python API)<br> - Generate Final Map (GEE Python API)<br>
(10)Accuracy Assessment<br> - Stratified Random Sampling by Land-Cover Class (GEE Python API)<br> -  Visual Evaluation of Random Samples (Google Earth)<br> - Evaluation of the Contingency Matrix (GEE Python API)



<img src="./imgs/JensenClfProcess.PNG">

# Classification Scheme Definition

The classes and the detail of a classification scheme of a land cover map are determined by its intended use.  A classification scheme can have multiple levels of detail.  It is good practice to design a classification scheme with mutually exclusive and exhaustive classes of either land cover OR land use.  For different levels or scales the class details need to be determined in a way that they can be easily aggregated such as in a hierarchical structure.  

The classification scheme used in the runoff factor map delineation in the CAPRA flood model consists of a mixture of detailed land cover and land use classes as described in Table 1.  This is not an optimal scenario as overlap of classes can lead to non-exclusiveness of classes (e.g., impermeable surface and paved roads).  It is not consistent in the level of the hierarchical structure either.  The two level one forest land cover classes in the classification scheme are “natural forests” differentiated from “forests” which at the second level are further divided into  seeded or cultivated.  Further “cropped furrows”,” cereals”, “leguminous” and their subclasses can be grouped under agricultural or cropland at the first level. Roads, rangeland and rest (uncultivated) are pure land use classes, with class “rangeland” overlapping with class “pasture”.  

In general it is a lot more difficult to derive land use from a remotely sensed image than it is to detect land cover, as land cover describes the surface material and therefore has a direct linkage to the spectral reflectance behavior of the material (e.g. land cover grassland vs. land use classes pasture, football field, city park).  Land use classification of detailed agricultural classes as suggested by the CAPRA classification scheme requires a very good knowledge and expertise of a region’s agricultural practices.  

An important component of a land-cover classification procedure is the proper choice and delineation of training sites, used to train the computer in pattern recognition.  It is crucial to develop a database of reference points with reliable ground cover or use information at the spatial scale and thematic detailed at which the mapping procedure will be performed.  Preferably such reference points are acquired in the field (in situ) or from high resolution aerial photography.  For each of the different land cover types of interest a set of representative samples would be collected using a GPS (or digitizing technique).  

In our case we do not have a database of ground reference points or aerial photography, and we do not have an intimate knowledge of agricultural practices of the region, therefore we will refer to the U.S. Geological Survey Land Use/Land Cover Classification System for use with Remote Sensor Data (Tbl. 2).  

<img src="./imgs/LCclassTable.PNG">

For this tutorial we will use classes based on level one of the USGS classification system.  The classes of interest for our mapping application are: Water, Forest Land, Rangeland, Agricultural Land, Barren Land, and Urban or Built-up Land.  
Since the classification scheme is a mix of land cover and land use we will rename classes with land use character to their corresponding land cover.  We also want to avoid spaces in the class names since we are going to use the names in code and spaces can be problematic.  The class names we are going to use instead are:

(1)	water 
(2)	forestLand
(3)	grassLand
(4)	cropLand
(5)	barrenLand
(6)	buildUpLand 

#### Question:
#### (1) Which of the level one classes of the U.S. Geological Survey Land Use/Land Cover Classification System for use with remote                  sensor Data do you expect to be present in your ROI (google it).

# Remote Sensing Process

One of the more common procedures to generate thematic land cover maps of a region is by applying pattern-recognition algorithms to satellite imagery in a digital image processing environment.  The use of remote sensing classification methods involve three general steps, (1) the selection and pre-processing of imagery adequate for the mapping of land cover classes of interest at their appropriate scales, (2) extraction and evaluation of spectral signatures and their statistical separability for all land cover classes of interest and development of a classification method and (3) evaluation and documentation of the mapping process and the final map product.   

(1)	Discrimination, and mapping of various land use/land cover classes depends on the spatial resolution (pixel size); the spectral resolution (number of bands or channels and their bandwidth); the radiometric resolution (range of discrete brightness values); and the temporal resolution (return time of the sensor) in relation to the classification scheme and minimum mapping unit or appropriate scale (Teillet et al., 1997 ,Rao et al., 2007).  Since the limitations of a mapping project are often constrained by its budget, the most economic approach to consider is to use data free of charge processed with open source image processing software, which is the a solution demonstrated in this tutorial.  

A large Landsat archive of 45 years of image acquisitions is available and open to the public. Landsat sensor specific information are accessible through the data portal of USGS (http://landsat.usgs.gov/Landsat_Search_and_Download.php).  For this exercise we are going to use Landsat 8 images. The Scan Line Corrector (SLC) in the Landsat 7 failed in in April of 2003 resulting in imagery with significant geometric error and considerable areas of no data value in each scene.  The Landsat 8 data source is adequate for the classes we identified in the classification scheme. Google Earth Engine (GEE) contains a variety of Landsat specific processing methods. Specifically, there are methods to compute at-sensor radiance, top-of-atmosphere reflectance, surface reflectance, cloud score and cloud-free composites.   
    
(2)	The extraction and evaluation of spectral signatures of all land cover classes of interest requires a training dataset.  In this exercise a set of training points for each class will be digitized on screen using the Google Earth application.  For the digitized sampling points spectral characteristics will then be extracted from the pre-processed imagery.  The variables we are interested in evaluating are reflectance values.  In addition we will consider derivative information, the Normalized Difference Vegetation Index (NDVI) and tasseled cap or Kauth-Thomas transformation which is based on the relationship of spectral characteristics to soil brightness, moisture content and vegetation cover.  The pre-processing and preparation of all data and the extraction of variable values for each sample of the training data set will be performed in Google Earth and GEE.

The classification process is guided by the evaluation of the separability of classes for different combinations of reflectance bands and other derived variables based on the training set.  Separability analysis and feature selection are frequently based on descriptive class parameters such as the class specific mean vector and variance-covariance matrices.  For this tutorial instead of using parametric measures to evaluate class-separability, we will evaluate the effectiveness of a recursive partitioning algorithm, a non-parametric classification procedure (see pg. 38 in landCoverClassification_Theory.pdf).  Performance of different classifiers is assessed for all evaluated models based on accuracy measures derived from confusion matrices of the classified training dataset.  This initial evaluation of accuracies has a bias towards overestimating accuracies, and will only be used to select the model with the best fit.  The analysis will provide two important components for the final classification, a list of variables that are most suitable to differentiate the classes of interest, and the decision rules or classification tree (the model or classifier), that will be applied to all pixels of the entire area of interest, the Dominican Republic.  This part of the analysis will be performed in GEE Python API.      

(3)	The final step of the mapping project is the accuracy assessment of the final classification results (the final map).  For this purpose we will use a stratified random sampling design (see pg. 45 in landCoverClassification_Theory.pdf) applied to the sampling frame of all classified image elements (pixels).  For lack of ground reference data we will use Google Earth (or Google maps) as reference source to determine classification accuracy.


# Data Acquisition and Processing Options

There are multiple websites from agencies around the world that will equip you with high quality satellite imagery, free of charge. Here we will introduce different acquisition and processing options for Landsat data. The land cover classification will be carried out using the cloud computing resources in GEE. 

Landsat data is collected continuously along a near vertical path as the satellite moves from north to south. The data is randomly divided into nominal scenes with a spacing of approximately 160km. Landsat data is collected in accordance with the World Wide Reference System (WRS), which has catalogued the world's land mass into 57,784 scenes, each 183 km wide by 170 km long (Jensen, 2005). These divisions facilitate querying satellite imagery over any portion of the world by specifying a nominal scene center designated by ‘path’ and ‘row’ numbers (Rocchio, June, 2011). When looking for a scene of a certain area, it can be obtained by using the path and row. If the path and row are not known, then the coordinates of the area of interest can be used. 

Several sites can be used to locate and obtain Landsat satellite imagery. Some recommended sites are  Global Visualization Viewer (GloVis) portal and Earth Explorer by USGS, these two are considered to have the most complete collection of Landsat data, spanning the entire time of the program, beginning in the early 1970’s. 

### Exploration: GloVis

The USGS GloVis site at(http://glovis.usgs.gov) distributes Landsat data, as well as some other imagery from ASTER, MODIS and some others. 

(1) Go to the GloVis portal. You will see the following browse image viewer:

<img src="./imgs/Glovis_3.png">

(2) Under the Data Set menu, select Landsat 8 (L8) OLI/TIRS C1 Level-1<br>
(3) Under the Metadata Filter enter the desired date range: January, 2018 to May, 2019<br>
(4) Set the cloud cover from 0 to 20 and click on Apply<br>

The number of available scenes that match the criteria will be shown under the Dataset menu.

(5) Expand the timeline tab and browse the available L8 scenes<br> 
(6) Open the metadata for the *LC08_L1TP_007047_20190108_20190130_01_T1* scene


#### <i>Question
#### <i>(1)	What are path and row for the images that cover your ROI?

The L8 scenes can be downloaded and visualized in a Geographical Information Systems software. In this tutorial we will use the cloud based data in GEE to visualize and process the L8 scenes. 

### Exploration: Google Earth Engine

GEE provides a javascript interface where users can manage their assets, create scripts and visualize results. This interface is available in [GEE] https://code.earthengine.google.com/ Although we will use the python API of GEE, in this part of the tutorial we will use the javascript interface to load data. Later on, we will learn how to do it programatically using GEE Python API.

#### Overview of GEE web interface

We can distinguish four main areas as the image below shows. 
1. Left panel. Here we have three tabs: <i>scripts</i>, <i>asset</i> and <i>docs</i>. 
2. Middle panel. A script editor.
3. Right panel. It consist of three tabs: <i>Inspector</i>, <i>console</i> and <i>tasks</i>. 
4. Map area. We can add visualize data it in this section or edit features.<br> 

<img src="./imgs/GEE.png">

#### Loading an image

GEE has a large amount of Earth data free available for researchers. We're going to work with a Landsat 8 (L8) image collection which has been processed to surface reflectance values (i.e. the processing has attempted to remove the atmospheric effects). We will load the Landsat scene over our area of interest(Dominican Republic).<br> 

To inspect a L8 image in the region of interest (ROI), we will define the ROI as a point, filter the image collection to get a scene with few clouds, and display information about the image in the console. <br>

(1) Search for Dominican Republic (DR) in the search bar and click the result to pan and zoom the map to the Dominican Republic<br>
(2) Use the geometry tools to make a point on the DR (exit the drawing tool when you're finished). Name the resultant point by clicking on the import name ('geometry' by default)<br>
(3) Search for 'Landsat 8 Surface Reflectance Tier 1' Image Collection. Name the import landsat<br>
(4) Filter the ImageCollection by date and location, sort by metadata property called 'CLOUD_COVER' and get the first image out of this sorted collection:

You can type the following Java code into the script editor in GEE. The Python API of GEE will be used for the remainder of the tutorial

// Note that we need to cast the result of first() to Image<br>
var image = ee.Image(landsat

>// Filter to get only images in the specified range<br>
.filterDate('2018-01-01', '2018-12-31')

>// Filter to get only images at the location of the point<br>
.filterBounds(point)

>// Sort the collection by a metadata property<br>
sort('CLOUD_COVER')

>// Get the first image out of this collection<br>
.first());

(5) Display a readable representationof the image by printing it to the console:<br>

//Print the image to the console<br>
print('A Landsat scene:', image);

<img src="./imgs/javaCode.PNG">

(6) Activate the console and observe that after the descroptive text, an object is displayed. Expand and explore the object by clicking the little triangle next to the image name to see more information stored in that object. Expand the properties and inspect the metadata of the image. This is where the CLOUD_COVER property you just used is stored.

Now that you are familiar with GEE's user interface you can explore it further at your own leisure. The next sections of the tutorial will be performed using GEE Python Application Programming Interface (API). 